In [2]:
from utils import *
import pandas as pd

In [6]:
dataset_file = "dataset_resisc45.csv"

num_epochs = 5
batch_size = 32

In [7]:
df = pd.read_csv(dataset_file)

In [8]:
df.head()

,imgpath,label,label_index,fold
0,/Users/cristianion/Desktop/satimg_data/NWPU-RE...,forest,0,2
1,/Users/cristianion/Desktop/satimg_data/NWPU-RE...,forest,0,2
2,/Users/cristianion/Desktop/satimg_data/NWPU-RE...,forest,0,1
3,/Users/cristianion/Desktop/satimg_data/NWPU-RE...,forest,0,4
4,/Users/cristianion/Desktop/satimg_data/NWPU-RE...,forest,0,4


In [9]:
num_classes = len(list(df['label'].unique()))

In [12]:
for i in range(num_classes):
    positives = np.sum(df['label_index'] == i)
    negatives = np.sum(df['label_index'] != i)
    print(positives, negatives)

700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800
700 30800


In [20]:
"""
Train pretrained resnet, with feature extract cross validation
"""
print_versions()
device = get_device()

df = pd.read_csv(dataset_file)
num_classes = len(list(df['label'].unique()))  # take number of classes from datatset
num_folds = len(list(df['fold'].unique())) 

pretrained_model_config = CNNParams(
    model_name=PretrainedModelsEnum.resnet,
    num_classes=num_classes,
    feature_extract=True,
    use_pretrained=True,
)

fold_stats = []
fold_train_losses = []
fold_val_losses = []

for val_fold in range(num_folds):
    model, input_size = get_pretrained_model(pretrained_model_config)
    print(model)
    model.to(device)

    dataset_train_config = DatasetConfig(
        dataset_file=dataset_file,
        transform=get_pretrained_transforms(input_size, DatasetTypeEnum.train),
        dataset_type=DatasetTypeEnum.train,
        val_fold=val_fold,
    )
    dataset_val_config = DatasetConfig(
        dataset_file=dataset_file,
        transform=get_pretrained_transforms(input_size, DatasetTypeEnum.val),
        dataset_type=DatasetTypeEnum.val,
        val_fold=val_fold,
    )

    train_set = DatasetClassification(dataset_train_config)
    val_set = DatasetClassification(dataset_val_config)

    train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

    print(f"Fold: {val_fold}\n------------------------------")

    criterion = nn.CrossEntropyLoss()
    params_to_update = get_params_requires_grad(model, pretrained_model_config.feature_extract)
    optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
    print(optimizer)

    train_losses = []
    val_losses = []  
    for t in range(num_epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train_loss = train_one_epoch(criterion, optimizer, train_dataloader, model, device)
        val_loss = val_one_epoch(criterion, val_dataloader, model, device)
        train_losses.append(train_loss)
        val_losses.append(val_loss)

        torch.save(model, f"models/resisc45_resnet_f{val_fold}_e{t+1}.pt")

        pd.DataFrame({
            'train_losses': train_losses,
            'val_losses': val_losses,
        }).to_csv(f"models/resisc45_resnet_f{val_fold}_e{t+1}_losses.txt")
    
#     eval_result = evaluate_classifier_multi(val_dataloader, model, device)
    
#     fold_stats.append(eval_result)
#     fold_train_losses.append(train_losses)
#     fold_val_losses.append(val_losses)

# avg_accuracy = np.mean([stat.accuracy for stat in fold_stats])
# avg_mse = np.mean([stat.mse for stat in fold_stats])

macOS-13.3.1-arm64-arm-64bit
PyTorch Version:  2.0.0
Torchvision Version:  0.15.1
Using mps device
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, 